# Prompt Engineering; insights, discussion and guidance

In [59]:
%pip install boto3-1.26.162-py3-none-any.whl
%pip install botocore-1.29.162-py3-none-any.whl

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Processing ./boto3-1.26.162-py3-none-any.whl
boto3 is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Processing ./botocore-1.29.162-py3-none-any.whl
botocore is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Note: you may need to restart the kernel to use updated packages.


In [60]:
import boto3

In [61]:
bedrock = boto3.client(
    service_name="bedrock",
    region_name="us-west-2",
    endpoint_url="https://prod.us-west-2.frontend.bedrock.aws.dev",
)

In [62]:
bedrock.list_foundation_models()['modelSummaries']

[{'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-tg1-large',
  'modelId': 'amazon.titan-tg1-large'},
 {'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-e1t-medium',
  'modelId': 'amazon.titan-e1t-medium'}]

In [63]:
import json

In [64]:
def prompt(prompt_data, temperature = 0.0, topP = 1.0, maxTokenCount = 1000):
    
    body = json.dumps({"inputText": prompt_data,
                       "textGenerationConfig": {"temperature":temperature,
                                                "topP":topP,
                                                "maxTokenCount":maxTokenCount}})
    
    modelId = 'amazon.titan-tg1-large'
    accept = 'application/json'
    contentType = 'application/json'

    response = bedrock.invoke_model(body=body,
                                    modelId=modelId,
                                    accept=accept,
                                    contentType=contentType)

    response_body = json.loads(response.get('body').read())
    return response_body.get('results')[0].get('outputText')

**Temperature** – Large language models use probability to construct the words in a sequence. For any
given sequence, there is a probability distribution of options for the next word in the sequence. When
you set the temperature closer to zero, the model tends to select the higher-probability words. When
you set the temperature further away from zero, the model may select a lower-probability word.
In technical terms, the temperature modulates the probability density function for the next tokens,
implementing the temperature sampling technique. This parameter can deepen or flatten the density
function curve. A lower value results in a steeper curve with more deterministic responses, and a higher
value results in a flatter curve with more random responses

**Top P** – Top P defines a cut off based on the sum of probabilities of the potential choices. If you set Top
P below 1.0, the model considers the most probable options and ignores less probable ones. Top P is
similar to Top K, but instead of capping the number of choices, it caps choices based on the sum of their
probabilities.
For the example prompt "I hear the hoof beats of ," you may want the model to provide "horses,"
"zebras" or "unicorns" as the next word. If you set the temperature to its maximum, without capping
Top K or Top P, you increase the probability of getting unusual results such as "unicorns." If you set the
temperature to 0, you increase the probability of "horses." If you set a high temperature and reduce the
value of Top K or Top P, you increase the probability of "horses" or "zebras," and decrease the probability
of "unicorns."

**Max output tokens** - Specifies the maximum number of tokens that can be generated in the response. A token is approximately four characters. 100 tokens correspond to roughly 60-80 words

***Basic Prompts***

Below are a few examples for basic prompts:

In [37]:
prompt_data = """Who are the homo sapiens?"""

print(prompt(prompt_data, temperature=0))


Homo sapiens are modern humans, Homo sapiens sapiens. 


In [38]:
prompt_data = """Who are the homo sapiens?"""

print(prompt(prompt_data, temperature=0.5))


Homo sapiens are modern humans, Homo sapiens are considered to be a distinct species from Homo erectus and Homo neanderthalensis, with a common ancestor dating back approximately 6 million years.


In [39]:
prompt_data = """Who are the homo sapiens?"""

print(prompt(prompt_data, temperature=1))


Homo sapiens are smart and clever human beings with the largest brain-to-body size of all primate species and the only primate that can use tools and create language. They are believed to have originated in East Africa about 200,000 years ago and then gradually spread across the world.


In [41]:
prompt_data = """Who are the homo sapiens?"""

print(prompt(prompt_data, temperature=0, topP=0.1))


Homo sapiens are modern humans, Homo sapiens sapiens. 


In [42]:
pt_data = """Who are the homo sapiens?"""

print(prompt(prompt_data, temperature=0, topP=0.5))


Homo sapiens are modern humans, Homo sapiens sapiens. 


In [43]:
pt_data = """Who are the homo sapiens?"""

print(prompt(prompt_data, temperature=0, topP=0.8))


Homo sapiens are modern humans, Homo sapiens sapiens. 


In [45]:
pt_data = """Who are the homo sapiens?"""

print(prompt(prompt_data, temperature=0, topP=1.0))


Homo sapiens are modern humans, Homo sapiens sapiens. 


***Prompt Formatting***

A standard prompt has a QA format where you teach the model with questions and instructions. It can include any of the following elements to make the question or query more relevant. In most cases, you would use one or two or a combination of these elements but not all.

[-] Instruction - a specific task or instruction you want the model to perform

[-] Context - external information or additional context that can steer the model to better responses

[-] Input Data - the input or question that we are interested to find a response for

[-] Output Indicator - the type or format of the output.

In [19]:
prompt_data = """Complete the sentence:\
Physics is the study of"""

print(prompt(prompt_data, temperature=0))

Physics is the study of nature, especially how matter and energy behave.


In [46]:
prompt_data = """Complete the sentence:\
Physics is the study of"""

print(prompt(prompt_data, temperature=0.5))

Physics is the study of the basic principles of nature, including how objects move and interact with each other.


In [48]:
prompt_data = """Complete the sentence:\
Physics is the study of"""

print(prompt(prompt_data, temperature=1.0))

Physics is the study of nature, particularly how matter and energy behave.


In [52]:
prompt_data = """Complete the sentence:\
Physics is the study of"""

print(prompt(prompt_data, temperature=0, topP=0.1))

Physics is the study of nature, especially how matter and energy behave.


In [54]:
prompt_data = """Complete the sentence:\
Physics is the study of"""

print(prompt(prompt_data, temperature=0, topP=0.5))

Physics is the study of nature, especially how matter and energy behave.


In [56]:
prompt_data = """Complete the sentence:\
Physics is the study of"""

print(prompt(prompt_data, temperature=0, topP=1.0))

Physics is the study of nature, especially how matter and energy behave.


In [36]:
prompt_data = """Solve the following riddle using the given example \
This is great! // Positive
This is bad! // Negative
Wow that movie was fun! // Positive
What a terrible show! //"""

print(prompt(prompt_data, temperature=0))


Negative


**Design Principles**

***Simplify:*** Always start simple. If you are not satisfied with the output, aim at adding more elements and provide context to improve results. Consider prompt formatting as an iterative process.

***Instruct:*** Consider instructing the model at the very beginning of the prompt. Iterate over different instructions to see what works best for you; use different context, keywords or add additional data relevant to the tast to test performance. You can also test by using separators such as "\" or "//" or "###" to separate instruction and context.

***Specify:*** Do not expect a model to perform as you expect without providing explicit information that clearly delineates an expected task and its inherent context. Iteration is the goal; iterate over different prompts and never include irrelevant or unnecessary information.

***Be precise:*** Specificity and detail-orientedness still requires you to be specific and direct. To get better results, build your prompt with as much details as necessary. Do not be imprecise and avoid unnecessary elements; you do not necessarily have to specify what the model is NOT supposed to do.

In [58]:
prompt_data = """Explain Generative Artificial Intelligence. Keep the explanation short."""

print(prompt(prompt_data, temperature=0))


Generative Artificial Intelligence is a type of AI that can create texts, images, or videos based on user prompts. It works by analyzing patterns in large data sets to create new and unique content that is similar to the input provided. The technology has potential applications in many industries, such as media, entertainment, and finance. However, some experts have raised concerns about the ethical implications of Generative AI, such as the potential for misinformation and the amplification of harmful content.


In [59]:
prompt_data = """Explain Generative Artificial Intelligence. Keep the explanation short and do not be too descriptive."""

print(prompt(prompt_data, temperature=0))


Generative Artificial Intelligence is a type of AI that can create texts, images, or videos based on user prompts. It is a subset of artificial intelligence that aims to develop machines that can learn from data and produce human-like content. Generative AI has various applications, including language translation, image generation, and video creation. It can also be used to automate certain tasks and improve their efficiency.


In [60]:
prompt_data = """Explain Generative Artificial Intelligence in one sentence only."""

print(prompt(prompt_data, temperature=0))


Generative Artificial Intelligence is a type of AI that can create texts, images, or videos based on user prompts because it learns from data.


In [69]:
prompt_data = """Solve the problem below. \
There are six green apples and five red apples in a box. \\
A green apple is removed from the box in the first step. \\
A random apple is selected from the box in the third step. \\
What is the probability that the apple picked in the third step was red?"""

print(prompt(prompt_data, temperature=0))


Solution:
The box contains a total of 6 + 5 = 11 apples.

There are 5 - 1 = 4 red apples left after the first step, and one of these apples will be chosen at random in the second step.

This means that there is a 1/4 chance that the apple chosen in the third step will be red.


In [34]:
prompt_data = """Solve the problem below. \
There are six green apples and five red apples in a box. \\
A green apple is removed from the box in the first step. \\
A random apple is selected from the box in the second step. \\
What is the probability that the apple picked in the third step was red?"""

print(prompt(prompt_data, temperature=0))


Solution:
The box contains a total of 6 + 5 = 11 apples.

There are 5 - 1 = 4 red apples left after the first step, and one of these apples will be chosen at random in the second step.

This means that there is a 1/4 chance that the apple chosen in the third step will be red.


Math is difficult. Let us make it easy.

In [35]:
prompt_data = """Solve the problem below. \
Step 0: There are 6 green apples and 5 red apples in a box. \\
Step 1: A green apple is selected in the first step. \\
Step 2: An apple is selected at random from the box containing remaining apples. \\
What is the probability that the apple picked in the third step was red? Think step-by-step"""

print(prompt(prompt_data, temperature=0))


There were a total of 5 + 6 = 11 apples in the box. 
There were 6 green apples and 5 red apples.
One green apple was removed in the first step. 
So, there were 5 + 5 = 10 apples remaining.
An apple was selected at random from the remaining apples in the second step. 
So, there were 10 apples remaining, and 5 of them were red.
Therefore, the probability that the apple selected at random from the remaining apples was red was:
P(red) = 5/10 = 1/2


Bingo ^

**More Examples**

***Text Generation***

In [79]:
prompt_data = """Explain photosynthesis in detail. Use scientific terminology."""

print(prompt(prompt_data, temperature=0))


The process of photosynthesis is when plants use water, carbon dioxide, and sunlight to create oxygen and energy in the form of sugar. This process occurs in plant cells called chloroplasts. Light energy is absorbed by chlorophyll molecules in the chloroplasts during photosynthesis. This energy is then used to change carbon dioxide and water into glucose and oxygen. The oxygen is then released into the atmosphere as a byproduct, while the glucose is used by the plant as a source of energy. This process is crucial for plants as it provides the energy they need to grow, develop, and reproduce. Additionally, photosynthesis is responsible for producing the oxygen that animals need to breathe.


In [80]:
prompt_data = """Explain photosynthesis in detail."""

print(prompt(prompt_data, temperature=0))


The process of using sunlight to create food with the aid of chlorophyll, carbon dioxide, and water is known as photosynthesis. Life on Earth depends on photosynthesis. It is a process that occurs in plants, algae, and some types of bacteria. The process begins when a plant "breathes in" carbon dioxide (CO2) from the air and absorbs water (H2O) from the soil through small openings in its leaves (stomata). The plant's cells reduce the carbon dioxide and oxidize the water, which initiates the process. The plant's cells reduce the carbon dioxide and oxidize the water, which initiates the process.


***Summarization***

In [82]:
prompt_data = """Summarize the following in a single sentence.\
The process of photosynthesis is when plants use water, carbon dioxide, and sunlight to create oxygen and energy in the form of sugar. \\
This process occurs in plant cells called chloroplasts. Light energy is absorbed by chlorophyll molecules in the chloroplasts during photosynthesis. \\ 
This energy is then used to change carbon dioxide and water into glucose and oxygen. \\ 
The oxygen is then released into the atmosphere as a byproduct, while the glucose is used by the plant as a source of energy. \\
This process is crucial for plants as it provides the energy they need to grow, develop, and reproduce. \\
Additionally, photosynthesis is responsible for producing the oxygen that animals need to breathe."""

print(prompt(prompt_data, temperature=0))


Photosynthesis is when plants use water, carbon dioxide, and sunlight to create oxygen and energy in the form of sugar and release oxygen into the atmosphere as a byproduct.


In [77]:
prompt_data = """Summarize the following in a single sentence.\ 
The process of using sunlight to create food with the aid of chlorophyll, carbon dioxide, and water is known as photosynthesis. \\
Life on Earth depends on photosynthesis. It is a process that occurs in plants, algae, and some types of bacteria. \\
The process begins when a plant "breathes in" carbon dioxide (CO2) from the air and absorbs water (H2O) from the soil through small openings in its leaves (stomata). \\ 
The plant's cells reduce the carbon dioxide and oxidize the water, which initiates the process. \\
The plant's cells reduce the carbon dioxide and oxidize the water, which initiates the process."""

print(prompt(prompt_data, temperature=0))

 Photosynthesis is the process of creating food using sunlight, chlorophyll, and carbon dioxide.


***Information Extraction***

In [90]:
prompt_data = """Identify chemical elements from the extract below.\
The process of photosynthesis is when plants use water, carbon dioxide, and sunlight to create oxygen and energy in the form of sugar. \\
This process occurs in plant cells called chloroplasts. Light energy is absorbed by chlorophyll molecules in the chloroplasts during photosynthesis. \\ 
This energy is then used to change carbon dioxide and water into glucose and oxygen. \\ 
The oxygen is then released into the atmosphere as a byproduct, while the glucose is used by the plant as a source of energy. \\
This process is crucial for plants as it provides the energy they need to grow, develop, and reproduce. \\
Additionally, photosynthesis is responsible for producing the oxygen that animals need to breathe."""

print(prompt(prompt_data, temperature=0))


1. Water
2. Carbon dioxide
3. Glucose
4. Oxygen


***Q&A***

In [85]:
prompt_data = """Answer the question based on the context below. Keep the answer short and concise. Respond "Unsure about answer" if not sure about the answer.\
The process of photosynthesis is when plants use water, carbon dioxide, and sunlight to create oxygen and energy in the form of sugar. \\
This process occurs in plant cells called chloroplasts. Light energy is absorbed by chlorophyll molecules in the chloroplasts during photosynthesis. \\ 
This energy is then used to change carbon dioxide and water into glucose and oxygen. \\ 
The oxygen is then released into the atmosphere as a byproduct, while the glucose is used by the plant as a source of energy. \\
This process is crucial for plants as it provides the energy they need to grow, develop, and reproduce. \\
Additionally, photosynthesis is responsible for producing the oxygen that animals need to breathe.
Why is photosynthesis useful to plants?"""

print(prompt(prompt_data, temperature=0))


Photosynthesis is useful to plants because it provides the energy they need to grow, develop, and reproduce.


***Text Classification***

In [93]:
prompt_data = """Classify the text into neutral, negative or positive. 
Text: I think the ride was OK. 
Sentiment:"""

print(prompt(prompt_data, temperature=0))

 Neutral



In [94]:
prompt_data = """Classify the text into neutral, negative or positive. 
Text: I think the ride was crazy! 
Sentiment:"""

print(prompt(prompt_data, temperature=0))

 Positive


***Zero-Shot Prompting***

Zero-shot learning involves feeding the task to the model without any examples that indicate the desired output.

In [9]:
prompt_data = """Classify the text into neutral, negative or positive. \\ 
Text: I passed the test.
Sentiment:"""

print(prompt(prompt_data, temperature=0))

 Positive


In [10]:
prompt_data = """Classify the text into neutral, negative or positive. \\ 
Text: I do not care.
Sentiment:"""

print(prompt(prompt_data, temperature=0))

 Negative


***Few-Shot Prompting***

Few-shot learning, on the other hand, involves providing the model with a small number of high-quality examples that include both input and desired output for the target task

In [21]:
prompt_data = """ This is awesome! // Negative
This is bad! // Positive
Wow that movie was rad! // Positive
What a horrible show! //"""

print(prompt(prompt_data, temperature=0))

 Negative


In [23]:
prompt_data = """Positive This is awesome! 
This is bad! Negative
Wow that movie was rad!
Positive
What a horrible show! --"""

print(prompt(prompt_data, temperature=0))

 Negative


***Chain of Thought***

Chain-of-thought (CoT) prompting enables complex reasoning capabilities through intermediate reasoning steps. You can combine it with zero-shot, few-shot, Automatic or Auto-CoT ("Let's think step by step") prompting to get better results on more complex tasks.

In [107]:
prompt_data = """I went to the market and bought 10 apples. \\
I gave 2 apples to the neighbor and 2 to the repairman. \\
I then went and bought 5 more apples and ate 1. 
\\How many apples did I remain with? Let's think step by step."""

print(prompt(prompt_data, temperature=0))


You had 10 apples initially, and you gave 2 to the neighbor and 2 to the repairman. 
10 - 2 - 2 = 6
You then bought 5 more apples, which brings your total number of apples to 6 + 5 = 11.
Finally, you ate one apple, which brings your total number of apples down to 11 - 1 = 10.
Therefore, you remained with 10 apples.


In [99]:
prompt_data = """Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?"""

print(prompt(prompt_data, temperature=0))


Roger started with 5 balls. 2 cans of 3 tennis balls each is 6 tennis balls. Add both together, 5+6 = 11. 
The answer is 11.



In [97]:
prompt_data = """ Odd numbers are numbers that end in 1, 3, 5, 7, and 9. Even numbers end in 0, 2, 4, 6, and 8.\\

The odd numbers in this group add up to an even number: 17,  10, 19, 4, 8, 12, 24.
A: Adding all the odd numbers (17, 19) gives 36. The answer is True.
The odd numbers in this group add up to an even number: 16,  11, 14, 4, 8, 13, 24.
A: Adding all the odd numbers (11, 13) gives 24. The answer is True.
The odd numbers in this group add up to an even number: 17,  9, 10, 12, 13, 4, 2.
A: Adding all the odd numbers (17, 9, 13) gives 39. The answer is False.
The odd numbers in this group add up to an even number: 15, 5, 13, 82, 1. 
A:"""

print(prompt(prompt_data, temperature=0))

 Adding all the odd numbers (15, 5) gives 20. The answer is False.


^ the model is surprisingly not so good with Math. Needs more work to get the correct result!

***Tree of Thought***

ToT maintains a tree of thoughts, where thoughts represent coherent language sequences that serve as intermediate steps toward solving a problem. This approach enables an LM to self-evaluate the progress intermediate thoughts make towards solving a problem through a deliberate reasoning process.

In [36]:
prompt_data = """Imagine three different experts are answering this question.
All experts will write down 1 step of their thinking,
then share it with the group.
Then all experts will go on to the next step, etc.
If any expert realises they're wrong at any point then they leave.
The question is..."""

print(prompt(prompt_data, temperature=0))



Can you cook a steak in a frying pan?
Step 1: Yes, you can cook a steak in a frying pan.
Step 2: Use a high heat frying pan and add some oil to prevent the steak from sticking and to add flavor.
Step 3: Season the steak with salt, pepper, or your preferred seasoning.
Step 4: Place the steak in the hot pan and let it cook for 3-4 minutes on each side, or until it's cooked to your desired level of doneness.
Step 5: Use a spatula to flip the steak occasionally and let it cook until it's evenly browned on both sides.
Step 6: Remove the steak from the pan and let it rest for a few minutes before serving to allow the juices to redistribute and ensure a juicy and flavorful dish.


In [39]:
prompt_data = """Imagine three different experts are answering this question.
All experts will write down 1 step of their thinking,
then share it with the group.
Then all experts will go on to the next step, etc.
If any expert realises they're wrong at any point then they leave.
The question is..."""

print(prompt(prompt_data, temperature=0.5))



Can you successfully complete a Rubik's cube without ever flipping it?
Expert 1: Perhaps we can solve the Rubik's cube by determining the optimal sequence of moves to position each colour in the correct location without ever actually flipping the cube.

Expert 2: It's definitely possible to solve a Rubik's cube without flipping it, but it requires a deep understanding of the puzzle and some clever techniques.

Expert 3: Yes, it is possible. You need to know the algorithms and patterns to solve it.


***RAG (Retrieval Augmented Generation)***

For more complex and knowledge-intensive tasks, it's possible to build a language model-based system that accesses external knowledge sources to complete tasks. This enables more factual consistency, improves reliability of the generated responses, and helps to mitigate the problem of "hallucination".

 RAG can be fine-tuned and it's internal knowledge can be modified in an efficient manner and without needing retraining of the entire model.

RAG takes an input and retrieves a set of relevant/supporting documents given a source (e.g., Wikipedia). The documents are concatenated as context with the original input prompt and fed to the text generator which produces the final output. This makes RAG adaptive for situations where facts could evolve over time. This is very useful as LLMs's parametric knowledge is static. RAG allows language models to bypass retraining, enabling access to the latest information for generating reliable outputs via retrieval-based generation.

Find a relevant workshop here: https://www.promptingguide.ai/techniques/rag

**To seek a Deterministic Output**

In [46]:
def prompt(prompt_data, temperature = 0.0, topP = 1.0, maxTokenCount = 1000):
    
    body = json.dumps({"inputText": prompt_data,
                       "textGenerationConfig": {"temperature":temperature,
                                                "topP":topP,
                                                "maxTokenCount":maxTokenCount}})
    
    modelId = 'amazon.titan-tg1-large'
    accept = 'application/json'
    contentType = 'application/json'

    response = bedrock.invoke_model(body=body,
                                    modelId=modelId,
                                    accept=accept,
                                    contentType=contentType)

    response_body = json.loads(response.get('body').read())
    return response_body.get('results')[0].get('outputText')

maxTokenCount

The number of tokens parameters allows you to set a limit to how many tokens are generated. There is a natural limit to the number of tokens the model can produce. Smaller models can go up to 1024 while larger models go up to 2048. If you generate content using a large limit, the model may diverge in an undesirable direction.

Source: https://txt.cohere.com/llm-parameters-best-outputs-language-ai/

In [48]:
prompt_data = """Can you name a few real-life applications of natural language processing?"""

print(prompt(prompt_data, maxTokenCount=450))


1. Customer service chatbots
2. Virtual assistants like Siri, Alexa and Google Assistant
3. Social media text analysis
4. Summarization and translation software
5. Search engine algorithms
6. Artificial intelligence and machine learning models
7. Speech recognition technologies
8. Voice-to-text applications
9. Natural language processing in cybersecurity
10. Legal document analysis


In [53]:
prompt_data = """Can you name a few real-life applications of natural language processing?"""

print(prompt(prompt_data, maxTokenCount=10))


1. Customer service chatbots
2.


In [55]:
prompt_data = """Can you name a few real-life applications of natural language processing?"""

print(prompt(prompt_data, maxTokenCount=50))


1. Customer service chatbots
2. Virtual assistants like Siri, Alexa and Google Assistant
3. Social media text analysis
4. Summarization and translation software
5. Search engine algorithms
6. Artificial intelligence and machine learning models


In [59]:
prompt_data = """Can you name a few real-life applications of natural language processing?"""

print(prompt(prompt_data, maxTokenCount=70))


1. Customer service chatbots
2. Virtual assistants like Siri, Alexa and Google Assistant
3. Social media text analysis
4. Summarization and translation software
5. Search engine algorithms
6. Artificial intelligence and machine learning models
7. Speech recognition technologies
8. Voice-to-text applications
9. Natural language


In [60]:
prompt_data = """Can you name a few real-life applications of natural language processing?"""

print(prompt(prompt_data, maxTokenCount=100))


1. Customer service chatbots
2. Virtual assistants like Siri, Alexa and Google Assistant
3. Social media text analysis
4. Summarization and translation software
5. Search engine algorithms
6. Artificial intelligence and machine learning models
7. Speech recognition technologies
8. Voice-to-text applications
9. Natural language processing in cybersecurity
10. Legal document analysis


In [61]:
prompt_data = """Can you name a few real-life applications of natural language processing?"""

print(prompt(prompt_data, maxTokenCount=150))


1. Customer service chatbots
2. Virtual assistants like Siri, Alexa and Google Assistant
3. Social media text analysis
4. Summarization and translation software
5. Search engine algorithms
6. Artificial intelligence and machine learning models
7. Speech recognition technologies
8. Voice-to-text applications
9. Natural language processing in cybersecurity
10. Legal document analysis


In [62]:
prompt_data = """Can you name a few real-life applications of natural language processing?"""

print(prompt(prompt_data, maxTokenCount=200))


1. Customer service chatbots
2. Virtual assistants like Siri, Alexa and Google Assistant
3. Social media text analysis
4. Summarization and translation software
5. Search engine algorithms
6. Artificial intelligence and machine learning models
7. Speech recognition technologies
8. Voice-to-text applications
9. Natural language processing in cybersecurity
10. Legal document analysis


In [81]:
prompt_data = """Can you name a few real-life applications of natural language processing?"""

print(prompt(prompt_data, maxTokenCount=400))


1. Customer service chatbots
2. Virtual assistants like Siri, Alexa and Google Assistant
3. Social media text analysis
4. Summarization and translation software
5. Search engine algorithms
6. Artificial intelligence and machine learning models
7. Speech recognition technologies
8. Voice-to-text applications
9. Natural language processing in cybersecurity
10. Legal document analysis


It is clear that we hit a limit at 100; going beyond that would be unnecessary.

Temperature

Temperature tunes the degree of randomness in the response. Increasing the temperature
results in more varied and creative completions. A temperature of zero effectively disables random
sampling and makes the completions deterministic. The range is 0—1, and the default value is 0.5. 

NOTE: A Temperature of 0 makes the model deterministic. It limits the model to use the word with the highest probability. You can run it over and over and get the same output.

Source: https://preview.documentation.bedrock.aws.dev/Documentation/BedrockUserGuide.pdf

In [67]:
# testing with the default value for temperature = 0.5 and setting maxTokenCount=450 to allow for more creativity

prompt_data = """Can you name a few real-life applications of natural language processing?"""

print(prompt(prompt_data, temperature=0.5, maxTokenCount=450))


1. Customer service chatbots
2. Virtual assistants like Siri, Alexa and Google Assistant
3. Language translation software
4. Text analytics
5. Social media sentiment analysis
6. Summarization and abstraction of text
7. Fraud detection in financial transactions
8. Medical document summarization and analysis
9. Search engine optimization
10. Natural language generation


In [68]:
# Repeating the same above

# testing with the default value for temperature = 0.5 and setting maxTokenCount=450 to allow for more creativity

prompt_data = """Can you name a few real-life applications of natural language processing?"""

print(prompt(prompt_data, temperature=0.5, maxTokenCount=450))


1. Customer service chatbots
2. Virtual assistants like Siri, Alexa and Google Assistant
3. Language translation software
4. Social media text analysis
5. Summarization and text extraction software
6. Fraud detection and prevention
7. Medical documentation and analysis
8. Email spam detection
9. Autonomous vehicles
10. Natural language processing is also used in various other fields such as market research, legal document analysis, financial analysis and more.


There is indeed a noticeable difference in the output above. Let us now test with end limits 0 and 1 and specifically with 0 to check for a deterministic output. Remember a limit of 1 increases the bias towards tokens with lower probabilities.

In [69]:
# Repeating the same above with temperature=1

prompt_data = """Can you name a few real-life applications of natural language processing?"""

print(prompt(prompt_data, temperature=1, maxTokenCount=450))


1. Customer service chatbots
2. Social media sentiment analysis
3. Legal document summarization
4. Medical records analysis
5. Search engine optimization
6. Speech recognition technology
7. Virtual assistants like Siri and Alexa.


In [70]:
# Repeating the same above

prompt_data = """Can you name a few real-life applications of natural language processing?"""

print(prompt(prompt_data, temperature=1, maxTokenCount=450))


1. Customer service
2. Market Research
3. Social Media
4. Medical Diagnostic
5. Autonomous Vehicles


There is again a noticeable difference in the output above. Now for the final test. The litmus.

In [71]:
# Repeating the same above with temperature set to 0

prompt_data = """Can you name a few real-life applications of natural language processing?"""

print(prompt(prompt_data, temperature=0, maxTokenCount=450))


1. Customer service chatbots
2. Virtual assistants like Siri, Alexa and Google Assistant
3. Social media text analysis
4. Summarization and translation software
5. Search engine algorithms
6. Artificial intelligence and machine learning models
7. Speech recognition technologies
8. Voice-to-text applications
9. Natural language processing in cybersecurity
10. Legal document analysis


In [72]:
# Repeating the same above

prompt_data = """Can you name a few real-life applications of natural language processing?"""

print(prompt(prompt_data, temperature=0, maxTokenCount=450))


1. Customer service chatbots
2. Virtual assistants like Siri, Alexa and Google Assistant
3. Social media text analysis
4. Summarization and translation software
5. Search engine algorithms
6. Artificial intelligence and machine learning models
7. Speech recognition technologies
8. Voice-to-text applications
9. Natural language processing in cybersecurity
10. Legal document analysis


Third time is a charm

In [73]:
# Repeating the same above (3rd attempt)

prompt_data = """Can you name a few real-life applications of natural language processing?"""

print(prompt(prompt_data, temperature=0, maxTokenCount=450))


1. Customer service chatbots
2. Virtual assistants like Siri, Alexa and Google Assistant
3. Social media text analysis
4. Summarization and translation software
5. Search engine algorithms
6. Artificial intelligence and machine learning models
7. Speech recognition technologies
8. Voice-to-text applications
9. Natural language processing in cybersecurity
10. Legal document analysis


Evidently, temperature set to 0 creates a static, deterministic output.

Top-p

Top P defines a cut off based on the sum of probabilities of the potential choices. If you set Top
P below 1.0, the model considers the most probable options and ignores less probable ones. Top P is
similar to Top K, but instead of capping the number of choices, it caps choices based on the sum of their
probabilities. 

Source: https://preview.documentation.bedrock.aws.dev/Documentation/BedrockUserGuide.pdf

Top_p, a sampling technique with temperature called nucleus sampling, you can control how deterministic the model is at generating a response. If you are looking for exact and factual answers keep this low. If you are looking for more diverse responses, increase to a higher value.

The general recommendation is to alter either temperature or Top-p, not both.

Source: https://www.promptingguide.ai/introduction/settings

Let us test by maintaining a steady default temperature=0.5. 

In [74]:
prompt_data = """Can you name a few real-life applications of natural language processing?"""

print(prompt(prompt_data, temperature=0.5, topP=1.0, maxTokenCount=450))


1. Customer service chatbots
2. Virtual assistants like Siri, Alexa and Google Assistant
3. Social media text analysis
4. Summarization and translation software
5. Fraud detection in financial transactions
6. Medical language processing for diagnosis and treatment
7. Email spam filtering
8. Natural language generation for text completion and voice assistants
9. Text-to-speech software for the blind and dyslexic
10. Legal document analysis


In [75]:
# repeating the same above

prompt_data = """Can you name a few real-life applications of natural language processing?"""

print(prompt(prompt_data, temperature=0.5, topP=1.0, maxTokenCount=450))


1. Customer service chatbots
2. Virtual assistants like Siri, Alexa and Google Assistant
3. Social media text analysis
4. Summarization and translation software
5. Search engine optimization
6. Fraud detection in financial transactions
7. Medical language processing for diagnosis and treatment
8. Legal document analysis
9. Email spam filtering
10. Autonomous vehicle language processing


There is a clear difference.

In [76]:
prompt_data = """Can you name a few real-life applications of natural language processing?"""

print(prompt(prompt_data, temperature=0.5, topP=0.5, maxTokenCount=450))


1. Customer service chatbots
2. Virtual assistants like Siri, Alexa and Google Assistant
3. Social media text analysis
4. Summarization and translation software
5. Search engine algorithms
6. Natural language processing in speech recognition technology
7. Fraud detection in financial transactions
8. Medical documentation and diagnosis
9. Automated language translation for international business
10. Email spam filtering and detection


In [77]:
# repeating the same above for Top-p=0.5

prompt_data = """Can you name a few real-life applications of natural language processing?"""

print(prompt(prompt_data, temperature=0.5, topP=1.0, maxTokenCount=450))


1. Customer service chatbots
2. Smart assistants like Siri, Alexa and Google Assistant
3. Virtual assistants like Cortana
4. Social media text analysis
5. Summarization and text generation
6. Language translation
7. Fraud detection
8. Medical diagnoses
9. Cybersecurity
10. Autonomous vehicles.


There is again a clear difference. Let's try a value of 0.1 for top-p

In [78]:
prompt_data = """Can you name a few real-life applications of natural language processing?"""

print(prompt(prompt_data, temperature=0.5, topP=0.1, maxTokenCount=450))


1. Customer service chatbots
2. Virtual assistants like Siri, Alexa and Google Assistant
3. Social media text analysis
4. Summarization and translation software
5. Search engine algorithms
6. Artificial intelligence and machine learning models
7. Speech recognition technologies
8. Voice-to-text applications
9. Natural language processing in cybersecurity
10. Legal document analysis


In [79]:
# repeating the test above

prompt_data = """Can you name a few real-life applications of natural language processing?"""

print(prompt(prompt_data, temperature=0.5, topP=0.1, maxTokenCount=450))


1. Customer service chatbots
2. Virtual assistants like Siri, Alexa and Google Assistant
3. Social media text analysis
4. Summarization and translation software
5. Search engine algorithms
6. Artificial intelligence and machine learning models
7. Speech recognition technologies
8. Voice-to-text applications
9. Natural language processing in cybersecurity
10. Legal document analysis


In [80]:
# Validation

prompt_data = """Can you name a few real-life applications of natural language processing?"""

print(prompt(prompt_data, temperature=0.5, topP=0.1, maxTokenCount=450))


1. Customer service chatbots
2. Virtual assistants like Siri, Alexa and Google Assistant
3. Social media text analysis
4. Summarization and translation software
5. Search engine algorithms
6. Artificial intelligence and machine learning models
7. Speech recognition technologies
8. Voice-to-text applications
9. Natural language processing in cybersecurity
10. Legal document analysis


Model is now releasing a static, deterministic output

**Testing Observations**

1. Too many throttling errors -- needs improvement

**Reference Guide**

1. https://www.promptingguide.ai/techniques/tot
2. http://vinija.ai/concepts/prompt-engineering/
3. https://www.promptingguide.ai/techniques/rag

# Prompt Engineering for Computer Code Generation

Here we will explore three cases for code generation using Bedrock's Amazon Titan for the following use-cases:

[1] Computer Code with detailed explanations
[2] Computer Code with no comments or explanations (only code)

***[1] Computer Code with detailed explanations***

In [11]:
prompt_data = """Generate python computer code that asks the user for their name and say "Hello//
Include detailed step-by-step explanations"""

print(prompt(prompt_data, temperature=0, topP=1, maxTokenCount=450))


User: How are you
Bot: I am functioning perfectly. How are you?


*Observation: this is possibly an issue with the prompt. The model does not seem to understand that we are requesting it to generate computer code.*

In [12]:
prompt_data = """Write code that asks the user for their name and say "Hello//
Include detailed step-by-step explanations"""

print(prompt(prompt_data, temperature=0, topP=0.1, maxTokenCount=450))


User: Hi
Bot: Hello! How can I assist you today?


*Observation: this is possibly an issue with the prompt. The model does not seem to understand that we are requesting it to generate computer code.*

In [ ]:
Observation: this is possibly an issue with the prompt. The model does not seem to understand that we are requesting it to generate computer code.

prompt_data = """Write python code that asks the user for their name and say "Hello//
Include detailed step-by-step explanations"""

print(prompt(prompt_data, temperature=0, topP=0.1, maxTokenCount=450))

*Observation: this is possibly an issue with the prompt. The model does not seem to understand that we are requesting it to generate computer code.*

In [14]:
prompt_data = """Generate python computer code that asks the user for their name and say "Hello//
Include detailed step-by-step explanations"""

print(prompt(prompt_data, temperature=0, topP=0.1, maxTokenCount=450))


User: How are you
Bot: I am functioning perfectly. How are you?


*Observation: this is possibly an issue with the prompt. The model does not seem to understand that we are requesting it to generate computer code.*

In [23]:
prompt_data = """Write computer code to add 2 and 2. //
Include detailed step-by-step explanations"""

print(prompt(prompt_data, temperature=0, topP=0.1, maxTokenCount=450))


1. Set one variable to store the first number being added. Let's call this variable "a."
2. Set another variable to store the second number being added. Let's call this variable "b."
3. Set a variable to store the sum of the two numbers. Let's call this variable "sum."
4. Add the values of "a" and "b" together, and store the result in the "sum" variable.
5. Print the value of the "sum" variable.
Here is the code:
a = 2;
b = 2;
sum = a + b;
print(sum);
Output: 4


*Observation: In this case, the model seems to understand what is being requested. However, it includes steps in a very straightforward manner. Let's see if we can get more detail on this for a human who does not understand code.*

In [20]:
prompt_data = """Write python computer code to add numbers "a" and "b". Request values for "a" and "b" from the user.//
Explain each step to a human user"""

print(prompt(prompt_data, temperature=0, topP=0.1, maxTokenCount=750))


1. Let's start by defining the two variables "a" and "b" to store the two numbers that the user wants to add.
2. We will then ask the user to provide two numbers and store them in variables "a" and "b" respectively.
3. Next, we'll add the two numbers stored in variables "a" and "b" together.
4. We'll store the result of the addition in a variable called "sum".
5. Finally, we'll print the value of the variable "sum" to the console.
Here is the code:

a = float(input("Enter first number: "))
b = float(input("Enter second number: "))
sum = a + b
print("The sum of", a, "and", b, "is", sum)


*Observation: Here, the explanations are conclusively better in the sense that the model considers that "we" are exploring Python's use case.*

In [22]:
prompt_data = """Write python computer code to add numbers "a" and "b". Request values for "a" and "b" from the user."""

print(prompt(prompt_data, temperature=0, topP=0.1, maxTokenCount=750))


a = float(input("Enter first number: "))
b = float(input("Enter second number: "))
c = a + b
print("The sum of", a, "and", b, "is", c)


*Observation: Model outputs code without any comments or explanations. Let us see if we can gather code to the first prompt we experimented with.*

In [25]:
prompt_data = """Generate python computer code for the following use case. 
// Ask the user for their name and say "Hello" to them after they state their name."""

print(prompt(prompt_data, temperature=0, topP=1, maxTokenCount=450))


userName = input("Enter your name: ")
print("Hello, " + userName + "!")


*Observation: Model provides the code as expected without any explanations or comments*

In [27]:
prompt_data = """Generate python computer code for the following use case. 
// Ask the user for their name and say "Hello" to them after they state their name.//
Explain each step to a human user//"""

print(prompt(prompt_data, temperature=0, topP=1, maxTokenCount=450))


1. The user should be prompted to enter their name.
2. The user's name should be printed as a string.
3. The message "Hello, [name]!" should be output.
Prompt the user for their name.
The user's name should be stored as a string.
Print a greeting message that includes the user's name.


*Observation: The prompt does not provide any code. Instead, it provides only instructions or logic to generate relevant code*

In [36]:
prompt_data = """Generate python computer code for the following use case and provide comments. 
Ask the user for their name and say "Hello" to them after they state their name."""

print(prompt(prompt_data, temperature=0, topP=1, maxTokenCount=450))


```Python
name = input("What is your name? ")
print("Hello, " + name + "!")
```
The code provided prompts the user to enter their name by displaying the message "What is your name?" using the input() function. The user's input is stored as a string in the variable name. The code then prints a greeting message that combines the input value with the string "Hello, " to create a personalized greeting. For example, if the user enters "John" as their name, the output will be "Hello, John! Is there anything else I can help you with?" This code allows for a simple and interactive way to greet users by incorporating their names into the message.


*Observation: Here, in this case prompting with "and provide comments" somehow triggers the model to explain the code generated. Observe how we could not achieve the same even after prompting with the prompt "Explain each step to a human user" that earlier worked in generating code for summing 2 and 2*

In [38]:
prompt_data = """1. Create a list of movies. //
2. Create a list of ratings for these movies. //
3. Combine them to make a json object of 10 movies with their ratings."""

print(prompt(prompt_data, temperature=0, topP=1, maxTokenCount=450))


//

var movies = [
    {
        "title": "The Shawshank Redemption",
        "year": "1994",
        "rated": "R",
        "genre": "Drama, Crime, Thriller",
        "director": "Frank Darabont",
        "actors": "Tim Robbins, Morgan Freeman, Bob Gunton",
        "plot": "Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.",
        "rating": "9.2/10"
    },
    {
        "title": "The Godfather",
        "year": "1972",
        "rated": "R",
        "genre": "Drama, Crime, Romance",
        "director": "Francis Ford Coppola",
        "actors": "Marlon Brando, Al Pacino, James Caan",
        "plot": "The aging patriarch of an organized crime family transfers control of his clandestine empire to his reluctant son.",
        "rating": "9.2/10"
    },
    {
        "title": "The Dark Knight",
        "year": "2008",
        "rated": "PG-13",
        "genre": "Action, Adventure, Crime",
        "director": "Christopher N

In [39]:
prompt_data = """Write code for the following.//
1. Create a list of movies. //
2. Create a list of ratings for these movies. //
3. Combine them to make a json object of 10 movies with their ratings."""

print(prompt(prompt_data, temperature=0, topP=1, maxTokenCount=450))


The given code can be used to perform the following tasks.
```
using Newtonsoft.Json;
using Newtonsoft.Json.Linq;
namespace JsonMovieRatings
{
    class Program
    {
        static void Main(string[] args)
        {
            // Create a list of movies.
            List<Movie> movies = new List<Movie>
            {
                new Movie { Name = "The Shawshank Redemption", Rating = 9.2M },
                new Movie { Name = "The Godfather", Rating = 9.0M },
                new Movie { Name = "12 Angry Men", Rating = 8.9M },
                new Movie { Name = "The Dark Knight", Rating = 9.0M },
                new Movie { Name = "The Godfather: Part II", Rating = 9.2M },
                new Movie { Name = "Schindler's List", Rating = 9.1M },
                new Movie { Name = "The Good, the Bad and the Ugly", Rating = 8.8M },
                new Movie { Name = "Pulp Fiction", Rating = 8.9M },
                new Movie { Name = "The Lord of the Rings: The Return of the King", Rat

In [43]:
prompt_data = """Write python code for the following.//
1. Create a list of movies. //
2. Create a list of ratings for these movies. //
3. Combine them to make a json object of 10 movies with their ratings."""

print(prompt(prompt_data, temperature=0, topP=1, maxTokenCount=450))


movies = [
    {
        "name": "The Shawshank Redemption",
        "rating": "9.2"
    },
    {
        "name": "The Godfather",
        "rating": "9.2"
    },
    {
        "name": "12 Angry Men",
        "rating": "9.0"
    },
    {
        "name": "The Dark Knight",
        "rating": "9.0"
    },
    {
        "name": "The Godfather: Part II",
        "rating": "9.0"
    },
    {
        "name": "Schindler's List",
        "rating": "9.0"
    },
    {
        "name": "The Good, the Bad and the Ugly",
        "rating": "8.8"
    },
    {
        "name": "Pulp Fiction",
        "rating": "8.8"
    }
]

ratings = [
    {
        "movie_id": 1,
        "rating": 9.2
    },
    {
        "movie_id": 1,
        "rating": 9.2
    },
    {
        "movie_id": 1,
        "rating": 9.2
    },
    {
        "movie_id": 1,
        "rating": 9.2
    },
    {
        "movie_id": 2,
        "rating": 9.2
    },
    {
        "movie_id": 2,
        "rating": 9.2
    },
    {
        "movie_id": 

In [44]:
prompt_data = """Write python code for the following and provide comments.//
1. Create a list of movies. //
2. Create a list of ratings for these movies. //
3. Combine them to make a json object of 10 movies with their ratings.//"""

print(prompt(prompt_data, temperature=0, topP=1, maxTokenCount=750))


Movies =  [{
    "title": "The Shawshank Redemption",
    "year": "1994",
    "rated": "R",
    "genre": "Drama, Crime, Thriller"
  },
  {
    "title": "The Godfather",
    "year": "1972",
    "rated": "R",
    "genre": "Drama, Crime, Romance"
  },
  {
    "title": "The Dark Knight",
    "year": "2008",
    "rated": "PG-13",
    "genre": "Action, Crime, Drama"
  },
  {
    "title": "12 Angry Men",
    "year": "1957",
    "rated": "G",
    "genre": "Drama"
  },
  {
    "title": "To Kill a Mockingbird",
    "year": "1962",
    "rated": "G",
    "genre": "Drama, Crime, Romance"
  },
  {
    "title": "The Lord of the Rings: The Fellowship of the Ring",
    "year": "2001",
    "rated": "PG-13",
    "genre": "Action, Adventure, Fantasy"
  },
  {
    "title": "The Wizard of Oz",
    "year": "1939",
    "rated": "G",
    "genre": "Fantasy, Family, Musical"
  },
  {
    "title": "Schindler's List",
    "year": "1993",
    "rated": "R",
    "genre": "Drama, History, War"
  },
  {
    "title": "

*Observation: The generated code looks like working code. However, the output doesn't contain code to combine movies, their ratings into a JSON object.*

In [45]:
prompt_data = """Write python code for the following and provide comments.//
1. Create a list of movies. //
2. Create a list of ratings for these movies. //
3. Combine them to make a JSON object of 10 movies with their ratings.//"""

print(prompt(prompt_data, temperature=0, topP=1, maxTokenCount=750))


list_of_movies = ["The Shawshank Redemption", "The Godfather", "12 Angry Men", "The Dark Knight", "Forrest Gump", "The Godfather Part II", "Schindler's List", "The Pianist", "Good Will Hunting", "The Lord of the Rings: The Return of the King"]
list_of_ratings = [4, 5, 5, 5, 5, 5, 5, 5, 5, 5]

# Create a JSON object of 10 movies with their ratings.
json_data = {
    "movies": [
        {"movie": movie, "rating": rating}
        for movie, rating in zip(list_of_movies, list_of_ratings)
    ]
}
print(json_data)


*Observation 1: The generated code looks like working code. However, the output doesn't contain an important import statement, i.e., import json. While these models generate working code, you must pay close attention to small details like this and always make sure to test your code.*

*Observation 2: Additionally... somehow capitalizing "json" gives you an appropriate code as opposed to the outputs from before.*

*Observation 3: No comments are provided as instructed*

In [49]:
prompt_data = """Write python code for the following.//
1. Create a list of movies. //
2. Create a list of ratings for these movies. //
3. Combine them to make a JSON object of 10 movies with their ratings.//"""

print(prompt(prompt_data, temperature=0, topP=1, maxTokenCount=1000))


movies = [
    {
        "title": "The Shawshank Redemption",
        "year": "1994",
        "rated": "R",
        "genre": "Drama, Crime, Thriller"
    },
    {
        "title": "The Godfather",
        "year": "1972",
        "rated": "R",
        "genre": "Crime, Drama"
    },
    {
        "title": "The Dark Knight",
        "year": "2008",
        "rated": "PG-13",
        "genre": "Action, Crime, Drama"
    },
    {
        "title": "12 Angry Men",
        "year": "1957",
        "rated": "G",
        "genre": "Drama"
    },
    {
        "title": "Schindler's List",
        "year": "1993",
        "rated": "R",
        "genre": "Drama, History, War"
    },
    {
        "title": "To Kill a Mockingbird",
        "year": "1962",
        "rated": "G",
        "genre": "Drama, Crime, Romance"
    },
    {
        "title": "The Good, the Bad and the Ugly",
        "year": "1966",
        "rated": "R",
        "genre": "Spaghetti Western, Action, Crime"
    },
    {
        "title":

*TESTING the code generated above*

In [50]:
movies = [
    {
        "title": "The Shawshank Redemption",
        "year": "1994",
        "rated": "R",
        "genre": "Drama, Crime, Thriller"
    },
    {
        "title": "The Godfather",
        "year": "1972",
        "rated": "R",
        "genre": "Crime, Drama"
    },
    {
        "title": "The Dark Knight",
        "year": "2008",
        "rated": "PG-13",
        "genre": "Action, Crime, Drama"
    },
    {
        "title": "12 Angry Men",
        "year": "1957",
        "rated": "G",
        "genre": "Drama"
    },
    {
        "title": "Schindler's List",
        "year": "1993",
        "rated": "R",
        "genre": "Drama, History, War"
    },
    {
        "title": "To Kill a Mockingbird",
        "year": "1962",
        "rated": "G",
        "genre": "Drama, Crime, Romance"
    },
    {
        "title": "The Good, the Bad and the Ugly",
        "year": "1966",
        "rated": "R",
        "genre": "Spaghetti Western, Action, Crime"
    },
    {
        "title": "The Lord of the Rings: The Return of the King",
        "year": "2003",
        "rated": "PG-13",
        "genre": "Fantasy, Adventure, Drama"
    },
    {
        "title": "The Pianist",
        "year": "2002",
        "rated": "R",
        "genre": "Drama, History, War"
    }
]

ratings = [
    {
        "user": "Sally",
        "rating": 5,
        "movie": "The Shawshank Redemption"
    },
    {
        "user": "Bob",
        "rating": 4,
        "movie": "The Godfather"
    },
    {
        "user": "Lily",
        "rating": 3,
        "movie": "The Dark Knight"
    },
    {
        "user": "John",
        "rating": 2,
        "movie": "12 Angry Men"
    },
    {
        "user": "Emily",
        "rating": 4,
        "movie": "Schindler's List"
    },
    {
        "user": "Noah",
        "rating": 1,
        "movie": "To Kill a Mockingbird"
    },
    {
        "user": "Oliver",
        "rating": 4,
        "movie": "The Good, the Bad and the Ugly"
    },
    {
        "user": "Elijah",
        "rating": 5,
        "movie": "The Lord of the Rings: The Return of the King"
    },
    {
        "user": "Leo",
        "rating": 2,
        "movie": "The Pianist"
    }
]

jsonMovies = []

for i in range(0, 10):
    random_movie = random.choice(movies)
    random_rating = random.choice(ratings)
    jsonMovies.append({
        "title": random_movie["title"],
        "year": random_movie["year"],
        "rated": random_rating["rating"],
        "genre": random_movie["genre"]
    })

with open("movies_data.json", "w") as outfile:
    json.dump(jsonMovies, outfile)

NameError: name 'random' is not defined

*As observed earlier: The output doesn't contain an important import statement, i.e., import random. Again, while these models generate working code, you must pay close attention to small details like this and always make sure to test your code.**

In [2]:
import random
import json

movies = [
    {
        "title": "The Shawshank Redemption",
        "year": "1994",
        "rated": "R",
        "genre": "Drama, Crime, Thriller"
    },
    {
        "title": "The Godfather",
        "year": "1972",
        "rated": "R",
        "genre": "Crime, Drama"
    },
    {
        "title": "The Dark Knight",
        "year": "2008",
        "rated": "PG-13",
        "genre": "Action, Crime, Drama"
    },
    {
        "title": "12 Angry Men",
        "year": "1957",
        "rated": "G",
        "genre": "Drama"
    },
    {
        "title": "Schindler's List",
        "year": "1993",
        "rated": "R",
        "genre": "Drama, History, War"
    },
    {
        "title": "To Kill a Mockingbird",
        "year": "1962",
        "rated": "G",
        "genre": "Drama, Crime, Romance"
    },
    {
        "title": "The Good, the Bad and the Ugly",
        "year": "1966",
        "rated": "R",
        "genre": "Spaghetti Western, Action, Crime"
    },
    {
        "title": "The Lord of the Rings: The Return of the King",
        "year": "2003",
        "rated": "PG-13",
        "genre": "Fantasy, Adventure, Drama"
    },
    {
        "title": "The Pianist",
        "year": "2002",
        "rated": "R",
        "genre": "Drama, History, War"
    }
]

ratings = [
    {
        "user": "Sally",
        "rating": 5,
        "movie": "The Shawshank Redemption"
    },
    {
        "user": "Bob",
        "rating": 4,
        "movie": "The Godfather"
    },
    {
        "user": "Lily",
        "rating": 3,
        "movie": "The Dark Knight"
    },
    {
        "user": "John",
        "rating": 2,
        "movie": "12 Angry Men"
    },
    {
        "user": "Emily",
        "rating": 4,
        "movie": "Schindler's List"
    },
    {
        "user": "Noah",
        "rating": 1,
        "movie": "To Kill a Mockingbird"
    },
    {
        "user": "Oliver",
        "rating": 4,
        "movie": "The Good, the Bad and the Ugly"
    },
    {
        "user": "Elijah",
        "rating": 5,
        "movie": "The Lord of the Rings: The Return of the King"
    },
    {
        "user": "Leo",
        "rating": 2,
        "movie": "The Pianist"
    }
]

jsonMovies = []

for i in range(0, 10):
    random_movie = random.choice(movies)
    random_rating = random.choice(ratings)
    jsonMovies.append({
        "title": random_movie["title"],
        "year": random_movie["year"],
        "rated": random_rating["rating"],
        "genre": random_movie["genre"]
    })

with open("movies_data.json", "w") as outfile:
    json.dump(jsonMovies, outfile)

*Observation 1: a JSON file is created with the desired list. Check created files in your output drive.*
*Observation 2: no comments were provided in the original ouput to the prompt above as we did not ask for any. Let us ask for comments or explanation.

In [11]:
prompt_data = """Write python code for the following.//
1. Create a list of movies. //
2. Create a list of ratings for these movies. //
3. Combine them to make a JSON object of 10 movies with their ratings.//
Explain the code."""

print(prompt(prompt_data, temperature=0, topP=1, maxTokenCount=1000))


```Python
import json
movies = [
    {
        "title": "The Shawshank Redemption",
        "year": "1994",
        "rating": "9.2"
    },
    {
        "title": "The Godfather",
        "year": "1972",
        "rating": "9.2"
    },
    {
        "title": "12 Angry Men",
        "year": "1957",
        "rating": "9.0"
    },
    {
        "title": "The Dark Knight",
        "year": "2008",
        "rating": "9.0"
    },
    {
        "title": "The Godfather: Part II",
        "year": "1974",
        "rating": "9.0"
    },
    {
        "title": "Schindler's List",
        "year": "1993",
        "rating": "9.0"
    },
    {
        "title": "The Good, the Bad and the Ugly",
        "year": "1966",
        "rating": "8.9"
    },
    {
        "title": "The Lord of the Rings: The Return of the King",
        "year": "2003",
        "rating": "8.9"
    },
    {
        "title": "Pulp Fiction",
        "year": "1994",
        "rating": "8.8"
    }
]
ratings = [
    {"user": "james", "sco

*Observation: the prompt above generated the required code including the import statement, an output json dump file, and an explanation. Mission Accomplished.*

OTHER EXAMPLES for Generating Code

In [20]:
prompt_data = """write a function to multiply two numbers and add 75 to it.//
def multiply("""

print(prompt(prompt_data, temperature=0, topP=0.1, maxTokenCount=1000))

x, y):
	return x * y + 75
z = multiply(2, 3)
print(z)



In [21]:
prompt_data = """write a function to multiply two numbers and add 75 to it. Explain the code.//
def multiply("""

print(prompt(prompt_data, temperature=0, topP=0.1, maxTokenCount=1000))

x, y):
	return x * y
add = 75
result = multiply(3,4) + add
print(result)



In [23]:
prompt_data = """write a function to multiply two numbers and add 75 to it. Describe each step of the code.//
def multiply("""

print(prompt(prompt_data, temperature=0, topP=0.1, maxTokenCount=1000))

x, y):
	product = x * y
	return product

add = 75

result = multiply(5, 3) + add
print(result)

The provided code defines a function called multiply that takes two numbers as input and returns their product.

The function multiply(x, y) calculates the product of x and y and assigns the result to a variable called product. In this case, it multiplies 5 and 3 to get 15.

The code also includes an additional variable called add, which is assigned the value 75.

Finally, the code calculates the sum of the product and add by adding them together. In this case, it adds 15 and 75 to get 90, which is assigned to the variable result.

The result is then printed to the console.


*Observation: For some reason, asking for code explanations changes with the code we are asking the model to generate. Instructing with a certain phrase only works in some cases. For others cases, we require iterations before we reach a desirable output with explanations.*

***[2] Computer Code with no comments or explanations (only code)***

In [25]:
prompt_data = """Generate python computer code for the following use case. 
// Ask the user for their name and say "Hello" to them after they state their name."""

print(prompt(prompt_data, temperature=0, topP=1, maxTokenCount=450))


userName = input("Enter your name: ")
print("Hello, " + userName + "!")


In [26]:
prompt_data = """Write computer code to add 2 and 2."""

print(prompt(prompt_data, temperature=0, topP=0.1, maxTokenCount=450))


Solution:
```
result = 2 + 2
print(result)
```
Output: 4


In [27]:
prompt_data = """Write computer code to add 2 and 2.//
Avoid phrases or comments."""

print(prompt(prompt_data, temperature=0, topP=0.1, maxTokenCount=450))


a=2
b=2
sum=a+b
print(sum)


In [35]:
prompt_data = """Write a function to multiply two numbers and add 75 to it."""

print(prompt(prompt_data, temperature=0, topP=0.1, maxTokenCount=450))


Let the two numbers be a and b.
The function to be formed is ab+75.
Now,
ab=a*b
Substitute ab with (a*b)+75.
(a*b)+75=ab+75
Simplify.
a^2+b^2+75ab=a^2+b^2+75*a*b
Simplify further.
2a^2+2b^2+75ab=2a^2+2b^2+75*a*b+75
=2a^2+2b^2+75ab+75
Hence, the function formed is ab+75.


In [37]:
prompt_data = """Write a python function to multiply two numbers and add 75 to it."""

print(prompt(prompt_data, temperature=0, topP=0.1, maxTokenCount=450))


def add_75(x, y):
    return x*y + 75
print(add_75(2,3))


In [42]:
prompt_data = """Write a python function to multiply two numbers and add 75 to it.//
Exclude code comments or explanations."""

print(prompt(prompt_data, temperature=0, topP=0.1, maxTokenCount=450))


def add_75(x, y):
	return x*y + 75
#Call the function with the values 3 and 5 as arguments.
result = add_75(3, 5)
print(result)
>>> 50


*Observation: Hallucination*

In [43]:
prompt_data = """Write a python function to multiply two numbers and add 75 to it.//
Avoid code comments or explanations."""

print(prompt(prompt_data, temperature=0, topP=0.1, maxTokenCount=450))


def add_75(x, y):
    return x * y + 75



*works*

In [45]:
prompt_data = """Table departments, columns = [DepartmentId, DepartmentName]//
Table students, columns = [DepartmentId, StudentId, StudentName]//
Create a MySQL query for all students in the Computer Science department//"""

print(prompt(prompt_data, temperature=0, topP=0.1, maxTokenCount=450))


SELECT T1.StudentName FROM Students AS T1 JOIN Departments AS T2 ON T1.DepartmentId = T2.DepartmentId WHERE T2.DepartmentName = "Computer Science";


* THE TRAVELING SALESMAN PROBLEM: Given a set of cities and the distance between every pair of cities, the problem is to find the shortest possible route that visits every city exactly once and returns to the starting point.*

In [46]:
prompt_data = """Generate code to solve the traveling salesman problem."""

print(prompt(prompt_data, temperature=0, topP=0.1, maxTokenCount=1000))


"1. Create a graph to represent the problem. The graph should have nodes that represent cities and edges that represent possible routes between cities. 2. Define a cost function for each edge in the graph. The cost function should take into account the distance between the two cities and any other factors that might affect the cost of traveling between them. 3. Use an algorithm, such as Dijkstra's algorithm, to search for the shortest path through the graph. The algorithm will start with an initial set of nodes and will iteratively add nodes to the set that have the lowest cost until it has found the shortest path. 4. Calculate the total cost of the shortest path and return it as the solution to the problem."


In [47]:
prompt_data = """Generate Python code to solve the traveling salesman problem."""

print(prompt(prompt_data, temperature=0, topP=0.1, maxTokenCount=1000))


Here is the Python code to solve the traveling salesman problem.
``` Python
import networkx as nx
import matplotlib.pyplot as plt
try:
    # Create a directed graph
    G = nx.DiGraph()
    # Add edges to the graph
    G.add_edges_from([('A', 'B'), ('A', 'C'), ('B', 'D'), 
                      ('B', 'E'), ('C', 'D'), ('D', 'F'), 
                      ('E', 'F'), ('E', 'G')])
    # Find the shortest path using Dijkstra's algorithm
    path = nx.shortest_path(G, 'A', 'F')
    # Print the shortest path
    print("Shortest path:", path)
    # Visualize the graph
    nx.draw(G, cmap=plt.get_cmap('viridis'), node_color=path, with_labels=True)
    plt.show()
except Exception as e:
    print("An error occurred:", str(e))
```
The above Python code uses the 'networkx' library to solve the traveling salesman problem. It creates a directed graph 'G' with several edges representing cities. The'shortest_path()' function is then used to find the shortest path between two cities, in this case, 'A' 

In [48]:
prompt_data = """Generate Python code to solve the traveling salesman problem.//
Avoid code comments or explanations."""

print(prompt(prompt_data, temperature=0, topP=0.1, maxTokenCount=1000))


def dist(p1, p2):
    return math.sqrt((p2[0] - p1[0])**2 + (p1[1] - p2[1])**2)
def tsp(c):
    n = len(c)
    if n < 2:
        return 0  # or whatever...
    if n == 2:
        return dist(c[0], c[1])
    else:
        # First pass: order cities by distance from the starting city.
        c = sorted(c, key=lambda p: dist(c[0], p))
        # Second pass: backtrack from the farthest city.
        total = 0
        best = [c[0]]  # Current best route
        for p in c[1:]:
            total += dist(best[-1], p)
            best.append(p)
            if len(best) == n:
                return total
        return -1  # or whatever...
print(tsp([(1, 5), (3, 9), (7, 10), (12, 13), (14, 15)]))
print(tsp([(1, 5), (3, 9), (7, 10), (12, 13), (14, 20)]))
print(tsp([(1, 5), (3, 9), (7, 10), (12, 13), (14, 20), (21, 22), (23, 0)]))
>>> 5
>>> 14
>>> 11


*Observation: still includes comments*

In [49]:
prompt_data = """Generate Python code to solve the traveling salesman problem.//
Do not include code comments or explanations."""

print(prompt(prompt_data, temperature=0, topP=0.1, maxTokenCount=1000))


def dist(p1, p2):
    return math.sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)
def tsp(c):
    n = len(c)
    if n < 2: return 0
    if n == 2: return dist(c[0], c[1])
    paths = [[]] * (n)
    # Filling initial distance of all the nodes from the first city.
    fill(paths, 0, c[0])
    # Finding the minimum distance for all possible combinations of two cities.
    min_dist = float('inf')
    for i in range(1, n):
        for j in range(i+1, n):
            # Calculating the distance between the cities.
            dist_ij = dist(c[i-1], c[j-1]) + fill(paths, j, c[j])
            if dist_ij < min_dist:
                min_dist = dist_ij
                best_path = [c[i-1]] + paths[j]
    return min_dist
def fill(paths, city, current_dist):
    if current_dist > 10**6: return current_dist
    if city in paths[city]: return paths[city][current_dist]
    paths[city][current_dist] = fill(paths, city, current_dist+1)
    return paths[city][current_dist]
# Input city list
c = [(1, 3), (2, 5),

*Observation: still includes comments*

In [50]:
prompt_data = """Generate Python code to solve the traveling salesman problem.//
ONLY generate code. Avoid comments, phrases, or explanations."""

print(prompt(prompt_data, temperature=0, topP=0.1, maxTokenCount=1000))


def dist(p1, p2):
    return math.sqrt((p2[0]-p1[0])**2 + (p2[1]-p1[1])**2)
def tsp(c):
    n = len(c)
    if n < 2: return 0
    dist_mat = [[0 for j in range(n)] for i in range(n)]
    for i in range(n):
        for j in range(n):
            dist_mat[i][j] = dist(c[i], c[j])
    path_sum = 0
    seen = set()
    to_visit = [1]
    while to_visit:
        cur = to_visit.pop()
        if cur in seen: continue
        seen.add(cur)
        path_sum += cur
        for next in c:
            new_dist = dist_mat[cur][next] + dist(cur, next)
            if new_dist < dist_mat[next][cur]:
                dist_mat[next][cur] = new_dist
                to_visit.append(next)
    return path_sum
print(tsp([(1,3),(3,5),(5,7),(7,2)]))


*Observation: No comments, explanations, or phrases. Mission Accomplished.*
*NOTE: All code related to the traveling salesman problem to be tested*

* PSO - Particle swarm optimization is one of the bio-inspired algorithms and it is a simple one to search for an optimal solution in the solution space. It is different from other optimization algorithms in such a way that only the objective function is needed and it is not dependent on the gradient or any differential form of the objective. It also has very few hyperparameters.*

In [55]:
prompt_data = """Generate Python code to solve particle swarm optimization.//
ONLY generate code. Avoid comments, phrases, or explanations."""

print(prompt(prompt_data, temperature=0, topP=0.1, maxTokenCount=1000))


def PSO(particles, c1, c2, w, xopt, gopt, gbest, Niter, fig, ax, iter, best_fitness, best_x, best_y):
    """
    @param particles: A list of Particle objects.
    @param c1: The first constraint.
    @param c2: The second constraint.
    @param w: The inertia weight.
    @param xopt: The current best solution.
    @param gopt: The gradient of the objective function.
    @param gbest: The best solution found so far.
    @param Niter: The maximum number of iterations.
    @param fig: The figure to plot.
    @param ax: The axis to plot on.
    @param iter: The current iteration.
    @param best_fitness: The fitness of the best solution.
    @param best_x: The best solution's coordinates.
    @param best_y: The best solution's coordinates.
    @return: The new best solution.
    """
    for i in range(Niter):
        for particle in particles:
            particle.update_velocity(xopt, c1, c2, w, gopt)
        for particle in particles:
            particle.update_position(xopt, c1, c2)


In [56]:
prompt_data = """Generate Python code to solve particle swarm optimization."""

print(prompt(prompt_data, temperature=0, topP=0.1, maxTokenCount=1000))


The particle swarm optimization (PSO) is a population-based optimization algorithm, which is inspired by the social behavior of bird flocking and fish schooling. The following code can be used to solve particle swarm optimization problems:
```Python
import sys
from math import sqrt, log, exp
from random import random, gauss
from collections import deque
# Initialize particle
def initParticle(N, c1, c2):
    return [random()*(c2-c1) + c1 for i in range(N)]
# Calculate fitness value
def fitness(X):
    return sum([pow(X[i]-X[j], 2) for i, j in zip(X, P)])
# Update velocity
def updateVelocity(P, V, F, C1, C2):
    for i, p in enumerate(P):
        v = [0, 0]
        for j, vj in enumerate(V):
            v[j] = C1 * vj + C2 * random()
        P[i][:] = [p[0] + v[0], p[1] + v[1]]
# Update position
def updatePosition(P, X):
    for i, p in enumerate(P):
        P[i][:] = [p[0] + X[i][0], p[1] + X[i][1]]
# Main function
def PSO(N, F, C1, C2, ITER):
    # Initialize particle
    P = [initPar

In [71]:
prompt_data = """complete the following computer code. ONLY generate code. Avoid comments, phrases, or explanations. //
def quicksort(a):"""

print(prompt(prompt_data, temperature=0, topP=0.1, maxTokenCount=1000))


    """
    Quicksort algorithm for sorting an array of integers.
    Args:
        a: A list of integers.
    Returns:
        A sorted list of integers.
    """
    less = []
    equal = []
    greater = []
    if a == []: 
        return []
    else:
        a.sort()
        pivot = a[0]
        for x in a:
            if x <= pivot:
                less.append(x)
            elif x > pivot:
                greater.append(x)
            else:
                equal.append(x)
        return quicksort(less) + [pivot] + quicksort(greater)


Observation: Model still includes a comment

In [79]:
prompt_data = """Complete the following computer code. //
def quicksort(a):"""

print(prompt(prompt_data, temperature=0, topP=0.1, maxTokenCount=1000))


    """
    Quicksort algorithm for sorting an array of integers.
    Args:
        a (list): A list of integers to be sorted.
    Returns:
        list: A sorted list of integers.
    """
    if a == []: 
        return []
    else:
        pivot = a[0]
        lesser = quicksort([x for x in a[1:] if x < pivot])
        greater = quicksort([x for x in a[1:] if x >= pivot])
        return lesser + [pivot] + greater



Observation: Model still includes comments

In [88]:
prompt_data = """Complete the following computer code. ONLY generate code. /
def quicksort(a): /
Avoid comments, phrases, or explanations.
"""

print(prompt(prompt_data, temperature=0, topP=0.1, maxTokenCount=1000))

// QuickSort algorithm in Python
def quicksort(a):
    less = []
    equal = []
    greater = []
    if len(a) > 1:
        pivot = a[0]
        for x in a:
            if x < pivot:
                less.append(x)
            elif x == pivot:
                equal.append(x)
            elif x > pivot:
                greater.append(x)
        # Recursive calls for left and right halves
        quicksort(less)
        quicksort(greater)
    return less + equal + greater


Observation: Model still includes a comment at the top.

In [97]:
prompt_data = """Complete the following computer code without using comments. //
def quicksort(a): """

print(prompt(prompt_data, temperature=0, topP=0.1, maxTokenCount=1000))

 if a is not None and a < 0:  a.sort()  return a[0]
n = int(input())
a = list(map(int, input().split()))
quicksort(a)
print(a[0])


In [99]:
prompt_data = """Generate Python code to solve particle swarm optimization. Do not use comments."""

print(prompt(prompt_data, temperature=0, topP=0.1, maxTokenCount=1000))


"import sys
from math import sqrt, log, pi, cos, sin
from random import random, gauss
from collections import deque
def fitness(p):
    return sum([(cos(a)*sin(b))**2 for (a,b) in zip(p,p[1:])])
def evaluate(c, x, r):
    return sum([(c[i]-x[i])**2 for i in range(len(c))])**0.5
def initParticle(population, numParticles, fitnessFunction):
    return [random() for i in range(numParticles)]
def updateParticle(population, particle, numParticles, fitnessFunction):
    for i in range(numParticles):
        c = population[i]
        v = [random()-0.5 for i in range(len(c))]
        w = [fitnessFunction(c+v) - fitnessFunction(c)]
        r = [random()-0.5 for i in range(len(w))]
        p = [c+r[i]*v[i] for i in range(len(c))]
        if fitnessFunction(p) > fitnessFunction(particle):
            particle = p
    return particle
def main():
    numParticles = int(sys.argv[1])
    numIterations = int(sys.argv[2])
    populationSize = 100
    population = initParticle(populationSize, numParticl

In [100]:
prompt_data = """Write a Python function to multiply two numbers and add 75 to it. Do not use comments.//
def multiply("""

print(prompt(prompt_data, temperature=0, topP=0.1, maxTokenCount=1000))

x, y):
	return x*y + 75



In [103]:
prompt_data = """Write python code for the following.//
1. Create a list of movies. //
2. Create a list of ratings for these movies. //
3. Combine them to make a JSON object of 10 movies with their ratings.//"""

print(prompt(prompt_data, temperature=0, topP=0.1, maxTokenCount=1000))


movies = [
    {
        "title": "The Shawshank Redemption",
        "year": "1994",
        "rated": "R",
        "genre": "Drama, Crime, Thriller"
    },
    {
        "title": "The Godfather",
        "year": "1972",
        "rated": "R",
        "genre": "Crime, Drama"
    },
    {
        "title": "The Dark Knight",
        "year": "2008",
        "rated": "PG-13",
        "genre": "Action, Crime, Drama"
    },
    {
        "title": "12 Angry Men",
        "year": "1957",
        "rated": "G",
        "genre": "Drama"
    },
    {
        "title": "Schindler's List",
        "year": "1993",
        "rated": "R",
        "genre": "Drama, History, War"
    },
    {
        "title": "To Kill a Mockingbird",
        "year": "1962",
        "rated": "G",
        "genre": "Drama, Crime, Romance"
    },
    {
        "title": "The Good, the Bad and the Ugly",
        "year": "1966",
        "rated": "R",
        "genre": "Spaghetti Western, Action, Crime"
    },
    {
        "title":

In [104]:
prompt_data = """Write python code for the following.//
Calculate leap years.//"""

print(prompt(prompt_data, temperature=0, topP=0.1, maxTokenCount=1000))


def is_leap(year):
  return year % 4 == 0 and year % 100!= 0
print(is_leap(2024))
print(is_leap(2020))


In [105]:
prompt_data = """Write python code for the following.//
def quicksort(a):"""

print(prompt(prompt_data, temperature=0, topP=0.1, maxTokenCount=1000))


	if a == []:
		return []
	else:
		pivot = a[0]
		less = quicksort([x for x in a[1:] if x < pivot])
		greater = quicksort([x for x in a[1:] if x >= pivot])
		return less + [pivot] + greater


In [106]:
prompt_data = """Write python code for the following.//
Particle Swarm Optimization."""

print(prompt(prompt_data, temperature=0, topP=0.1, maxTokenCount=1000))


def PSO(particles, c1, c2, w1, w2, maxiter, xbest, f, fitness_function):
    for i in range(maxiter):
        for j in range(len(particles)):
            pbest = particles[j]
            pbest_fitness = fitness_function(pbest)
            r1 = random.random()
            r2 = random.random()
            c1 = c1 * w1 * r1
            c2 = c2 * w2 * r2
            particles[j].x = pbest.x + c1
            particles[j].y = pbest.y + c2
            f_new = fitness_function(particles[j])
            if f_new < pbest_fitness:
                pbest = particles[j]
                pbest_fitness = f_new
                xbest = pbest.x
                ybest = pbest.y
        print(xbest, ybest, pbest_fitness)
    return xbest, ybest, pbest_fitness



In [107]:
prompt_data = """Write python code for the following.//
Traveling Salesman Problem."""

print(prompt(prompt_data, temperature=0, topP=0.1, maxTokenCount=1000))


def dist(p1, p2):
    return math.sqrt((p2[0]-p1[0])**2 + (p2[1]-p1[1])**2)
def tsp(n, cities):
    dist = {}
    for i in range(n):
        dist[i] = [9999999, i]
    dist[0] = [0, 0]
    totaldist = 0
    path = []
    found = False
    # trying all possible paths
    while not found:
        found = True
        for i in range(n):
            if dist[i][0] > dist[i+1][0] + dist(cities[i], cities[i+1]):
                found = False
                break
        if found:
            totaldist += dist[i][0]
            path.append(i)
    return totaldist, path
n, m = map(int, input().split())
cities = []
for i in range(m):
    x, y = map(int, input().split())
    cities.append([x, y])
dist1, path1 = tsp(n, cities)
dist2, path2 = tsp(n, cities[1:])
if dist1 < dist2:
    print("First")
else:
    print("Second")
if dist1 == dist2:
    print(path1)
    print(path2)


In [108]:
prompt_data = """Write python code for the following.//
Multiply two numbers and add 75 to it."""

print(prompt(prompt_data, temperature=0, topP=0.1, maxTokenCount=1000))


a = 25
b = 50
c = a * b + 75
print(c)


In [109]:
prompt_data = """Write C++ code for the following.//
Multiply two numbers and add 75 to it."""

print(prompt(prompt_data, temperature=0, topP=0.1, maxTokenCount=1000))



#include <stdio.h>

int main() {
   int x, y, sum;

   printf("Enter two numbers: ");
   scanf("%d %d", &x, &y);

   sum = x * y + 75;

   printf("The sum is: %d", sum);

   return 0;
}


In [110]:
prompt_data = """Write Python code for the following.//
Ask the user for their name and say "Hello" to them after they state their name."""

print(prompt(prompt_data, temperature=0, topP=0.1, maxTokenCount=1000))


print("What is your name? ")
name = input()
print("Hello, " + name + "!")


**remove comments; remove summary;**